# 15 아이유팬이 좋아할 만한 다른 아티스트 찾기

### 데이터 준비와 전처리

In [1]:
import os
import pandas as pd

rating_file_path='/content/drive/MyDrive/AIFFEL/EXP/data/recommend/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path='/content/drive/MyDrive/AIFFEL/EXP/data/recommend/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### 데이터 분석

In [5]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [6]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

movie데이터와 rating데이터를 movie_id로 묶음

In [7]:
data = pd.merge(ratings, movies, left_on='movie_id', right_on='movie_id')
data.head(10)

,user_id,movie_id,ratings,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
5,18,1193,4,978156168,One Flew Over the Cuckoo's Nest (1975),Drama
6,19,1193,5,982730936,One Flew Over the Cuckoo's Nest (1975),Drama
7,24,1193,5,978136709,One Flew Over the Cuckoo's Nest (1975),Drama
8,28,1193,3,978125194,One Flew Over the Cuckoo's Nest (1975),Drama
9,33,1193,5,978557765,One Flew Over the Cuckoo's Nest (1975),Drama


In [8]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = [ 'user_id','title', 'ratings']
data = data[using_cols]
data.head(10)

,user_id,title,ratings
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2,One Flew Over the Cuckoo's Nest (1975),5
2,12,One Flew Over the Cuckoo's Nest (1975),4
3,15,One Flew Over the Cuckoo's Nest (1975),4
4,17,One Flew Over the Cuckoo's Nest (1975),5
5,18,One Flew Over the Cuckoo's Nest (1975),4
6,19,One Flew Over the Cuckoo's Nest (1975),5
7,24,One Flew Over the Cuckoo's Nest (1975),5
8,28,One Flew Over the Cuckoo's Nest (1975),3
9,33,One Flew Over the Cuckoo's Nest (1975),5


In [9]:
data['title'] = data['title'].str.lower() 

1. rating에 있는 Unique한 영화 갯수.
2. rating에 있는 Unique한 사용자 수

In [10]:
#가장 인기있는 영화 30개(인기순)
movies_count = data.groupby('title')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [11]:
# 유저별 몇 편의 영화를 보고 있는지에 대한 통계
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

### 내가 선호하는 영화를 5가지 골라서 rating에 추가

In [12]:
# 내가 좋아하시는 영화의 데이터로 바꿔서 추가한다.
my_favorite = ['men in black (1997)' , 'fugitive, the (1993)' ,'terminator, the (1984)' ,
               'back to the future (1985)' ,'forrest gump (1994)']

# 'kimin'이라는 user_id가 위 영화의 평점을 5점씩 주었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['kimin']*5, 'title': my_favorite, 'ratings':[5]*5})

# user_id에 'kimin'이라는 데이터가 없다면
# 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
if not data.isin({'user_id':['kimin']})['user_id'].any():  
    data = data.append(my_playlist)   
    
data.tail(20) 

,user_id,title,ratings
836463,5328,outside ozona (1998),4
836464,5334,chain of fools (2000),3
836465,5334,song of freedom (1936),5
836466,5420,slappy and the stinkers (1998),3
836467,5433,nemesis 2: nebula (1995),3
836468,5494,smoking/no smoking (1993),4
836469,5556,modulations (1998),3
836470,5949,modulations (1998),5
836471,5675,broken vessels (1998),3
836472,5717,master ninja i (1984),4


### 모델에 활용하기 위한 전처리

In [13]:
# 고유한 유저, 타이틀을 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 타이틀 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [14]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['kimin'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(title_to_idx['forrest gump (1994)'])

6039
160


In [15]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# temp_title_data을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,ratings
0,0,0,5
1,1,0,5
2,2,0,4
3,3,0,4
4,4,0,5
...,...,...,...
0,6039,175,5
1,6039,141,5
2,6039,200,5
3,6039,22,5


### CSR Matrix

In [16]:
data['ratings']

0    5
1    5
2    4
3    4
4    5
    ..
0    5
1    5
2    5
3    5
4    5
Name: ratings, Length: 836483, dtype: int64

In [17]:
%%capture
 !pip install --upgrade pip setuptools 
 !pip install implicit==0.4.8

In [18]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.ratings, (data.user_id, data.title)), shape=(num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

### AlternatingLeastSquares 모델 작성 및 훈련

In [19]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [20]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [21]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [22]:
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
kimin

6039

In [ ]:
user_to_idx['kimin']

6039

In [ ]:
title_to_idx['back to the future (1985)']

22

In [23]:
kimin, back_to_the_future = user_to_idx['kimin'], title_to_idx['back to the future (1985)']
kimin_vector, back_to_the_future_vector = als_model.user_factors[kimin], als_model.item_factors[back_to_the_future]

In [24]:
kimin_vector

array([-0.21201964, -0.29460117,  0.0479541 , -0.5954896 ,  0.32943296,
        0.26828566,  0.57712734,  0.80095756, -0.18944928, -0.71026367,
        0.12059606, -0.9443594 , -0.22770004,  0.702014  ,  0.46916583,
       -0.30053836, -0.47834167,  0.33786562,  0.4973904 ,  0.24513985,
        0.3195721 , -0.7568489 , -0.29467583, -0.23529124, -0.25430456,
        0.57492685,  0.4956479 ,  0.39151314,  0.68383235, -0.02064993,
       -0.09887052, -0.0350383 , -0.30326292,  0.01089194,  0.16077164,
        0.24763249,  0.61534494, -0.14098287,  0.0902676 , -0.18985863,
        0.45190275,  0.3050754 ,  0.00502213,  0.15225574,  0.08036964,
       -0.34112626, -0.4487571 ,  0.06623311, -0.12068287,  0.5148448 ,
       -0.310267  , -0.81382966, -0.42051265, -0.16450989, -0.205621  ,
        0.5150511 ,  0.36530316, -0.35374364, -0.52812904, -0.9754539 ,
       -0.19961882, -0.33799046,  0.25192398,  0.2844024 , -0.08377905,
       -0.10428439, -0.02639477, -0.00600208,  0.11088596,  0.13

In [27]:
back_to_the_future_vector

array([-0.02736189, -0.01785918,  0.0373837 , -0.06319869,  0.01540845,
        0.02943425,  0.04635783,  0.0911158 , -0.00895115, -0.03858868,
        0.01119788, -0.04094511,  0.0123659 ,  0.00757806,  0.0294454 ,
       -0.01672099, -0.00910612,  0.01285143,  0.01498623, -0.00638257,
        0.01709565,  0.0118721 , -0.00727009,  0.00874756,  0.01875963,
        0.01582207,  0.03609428,  0.00356646,  0.00010147, -0.00382455,
        0.02002774,  0.02335032, -0.0092732 , -0.0120866 ,  0.03106358,
        0.02571313,  0.01660782, -0.00618139,  0.01454301,  0.01335419,
        0.02656299,  0.03563647, -0.02171374,  0.01277229, -0.00237009,
        0.01054659,  0.01832476,  0.02337857, -0.00528463,  0.0114892 ,
       -0.00064891,  0.00755183, -0.01037646,  0.02305074, -0.0034858 ,
        0.00453538,  0.01059404,  0.02196945,  0.01009165,  0.0218596 ,
       -0.00225857,  0.00898378,  0.04259533,  0.0371128 ,  0.01937127,
       -0.03078532,  0.01317824, -0.00842764,  0.01668153, -0.01

### 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [28]:
np.dot(kimin_vector, back_to_the_future_vector)

0.5624247

In [29]:
toy = title_to_idx['toy story (1995)']
toy_vector = als_model.item_factors[toy]
np.dot(kimin_vector, toy_vector)

0.19094457

### 내가 좋아하는 영화와 비슷한 영화를 추천받기

In [30]:
idx_to_title = {v:k for k,v in title_to_idx.items()}

def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [31]:
get_similar_title('men in black (1997)') 

['men in black (1997)',
 'jurassic park (1993)',
 'terminator 2: judgment day (1991)',
 'total recall (1990)',
 'independence day (id4) (1996)',
 'fifth element, the (1997)',
 'matrix, the (1999)',
 'lost world: jurassic park, the (1997)',
 'face/off (1997)',
 'galaxy quest (1999)']

In [32]:
get_similar_title('toy story (1995)') 

['toy story (1995)',
 'toy story 2 (1999)',
 "bug's life, a (1998)",
 'aladdin (1992)',
 'babe (1995)',
 'groundhog day (1993)',
 'lion king, the (1994)',
 'beauty and the beast (1991)',
 'pleasantville (1998)',
 "there's something about mary (1998)"]

In [33]:
get_similar_title('back to the future (1985)') 

['back to the future (1985)',
 "ferris bueller's day off (1986)",
 'when harry met sally... (1989)',
 'ghostbusters (1984)',
 'back to the future part ii (1989)',
 'big (1988)',
 'fish called wanda, a (1988)',
 'star wars: episode v - the empire strikes back (1980)',
 'e.t. the extra-terrestrial (1982)',
 'star wars: episode iv - a new hope (1977)']

### 내가 가장 좋아할 만한 영화들을 추천받기

In [34]:
user = user_to_idx['kimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
for i in title_recommended: 
    print("{} {:08.3f}".format(idx_to_title[i[0]], i[1]))

terminator 2: judgment day (1991) 0000.636
jurassic park (1993) 0000.583
matrix, the (1999) 0000.522
star wars: episode vi - return of the jedi (1983) 0000.434
total recall (1990) 0000.392
braveheart (1995) 0000.356
star wars: episode v - the empire strikes back (1980) 0000.341
die hard (1988) 0000.334
sixth sense, the (1999) 0000.331
star wars: episode iv - a new hope (1977) 0000.323
groundhog day (1993) 0000.317
hunt for red october, the (1990) 0000.308
speed (1994) 0000.288
saving private ryan (1998) 0000.270
raiders of the lost ark (1981) 0000.265
e.t. the extra-terrestrial (1982) 0000.244
independence day (id4) (1996) 0000.244
face/off (1997) 0000.239
aliens (1986) 0000.234
indiana jones and the last crusade (1989) 0000.217


In [35]:
rihanna = title_to_idx['die hard (1988)']
explain = als_model.explain(user, csr_data, itemid=rihanna)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('terminator, the (1984)', 0.22449578060611122),
 ('fugitive, the (1993)', 0.15380058664154078),
 ('back to the future (1985)', 0.021098516163633146),
 ('forrest gump (1994)', 0.003762971250068248),
 ('men in black (1997)', -0.07232804607483893)]

회고

1. 실행할 때마다 행렬 계산이므로 계산하고 나서 보면 저장된 부분이 있어서 다른 값이 나와 처음부터 다시 실행하였다.
2.  als_model.user_factors[kimin], als_model.item_factors[back_to_the_future] 부분에서 에러가 나왔는데 창환님의 도움으로 에러를 잡았다. ==>  !pip install --upgrade pip setuptools 
 !pip install implicit==0.4.8 : implicit 버전을 낮추었더니 에러가 안 나왔다.